In [10]:
from unsloth import FastLanguageModel
import torch

In [11]:
import torch
print(torch.cuda.is_available())  
print(torch.cuda.get_device_name(0))  


True
NVIDIA GeForce RTX 2060


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-7B-Instruct",
    max_seq_length=2048,
    load_in_4bit= False,
    dtype=None
)

==((====))==  Unsloth 2024.12.12: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.5.1+cu118. CUDA: 7.5. CUDA Toolkit: 11.8. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


d:\Envallconda\fine_tune_qwenfor_pic24_env\Lib\site-packages\accelerate\utils\modeling.py:1593: UserWarning: Current model requires 486539264 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing= "unsloth",
    random_state= 3047,
    use_rslora= False,
    loftq_config= None,
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.12.12 patched 28 layers with 28 QKV layers, 28 O layers and 0 MLP layers.


In [ ]:
import pandas as pd
from datasets import Dataset
alpaca_prompt = """You are a PIC24HJ32GP202 microcontroller expert answering student instruction. Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["response"]
    texts = []

    for instruction, output in zip(instructions, outputs):
        # Format the prompt without the input field
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

from datasets import load_dataset

file_path = "data.csv"
dataframe = pd.read_csv(file_path)
dataset = Dataset.from_pandas(dataframe)

# dataset = load_dataset("data.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)



Map: 100%|██████████| 634/634 [00:00<00:00, 869.59 examples/s] 


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map: 100%|██████████| 634/634 [00:00<00:00, 1790.58 examples/s]


NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

In [8]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 2060. Max memory = 6.0 GB.
4.262 GB of memory reserved.


In [9]:
trainer_stats = trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "give a sample code of defining input and output on a specific pin and connect the output to LED and set a timer to toggle that", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
decode = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decode[0])
#assume there is a system input to pin A0 with a wave with unknown frequency and the system will check if frequency bigger than 1kh or not if yes it will giv 1 as output else it will do nothing .output on Pin B3, measure the frequency input